In [2]:
import numpy as np
import pandas as pd
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from IPython.display import display, HTML
import re

In [3]:
data = pd.read_csv("recipes_crop.csv")

data.head()

,RecipeId,Name,AuthorId,AuthorName,CookTime,PrepTime,TotalTime,DatePublished,Description,Images,...,FatContent,SaturatedFatContent,CholesterolContent,SodiumContent,CarbohydrateContent,FiberContent,SugarContent,ProteinContent,RecipeServings,RecipeInstructions
0,38,Low-Fat Berry Blue Frozen Dessert,1533,Dancer,PT24H,PT45M,PT24H45M,1999-08-09T21:46:00Z,Make and share this Low-Fat Berry Blue Frozen ...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,2.5,1.3,8.0,29.8,37.1,3.6,30.2,3.2,4.0,"c(""Toss 2 cups berries with sugar."", ""Let stan..."
1,39,Biryani,1567,elly9812,PT25M,PT4H,PT4H25M,1999-08-29T13:12:00Z,Make and share this Biryani recipe from Food.com.,"c(""https://img.sndimg.com/food/image/upload/w_...",...,58.8,16.6,372.8,368.4,84.4,9.0,20.4,63.4,6.0,"c(""Soak saffron in warm milk for 5 minutes and..."
2,40,Best Lemonade,1566,Stephen Little,PT5M,PT30M,PT35M,1999-09-05T19:52:00Z,This is from one of my first Good House Keepi...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,0.2,0.0,0.0,1.8,81.5,0.4,77.2,0.3,4.0,"c(""Into a 1 quart Jar with tight fitting lid, ..."
3,41,Carina's Tofu-Vegetable Kebabs,1586,Cyclopz,PT20M,PT24H,PT24H20M,1999-09-03T14:54:00Z,This dish is best prepared a day in advance to...,"c(""https://img.sndimg.com/food/image/upload/w_...",...,24.0,3.8,0.0,1558.6,64.2,17.3,32.1,29.3,2.0,"c(""Drain the tofu, carefully squeezing out exc..."
4,42,Cabbage Soup,1538,Duckie067,PT30M,PT20M,PT50M,1999-09-19T06:19:00Z,Make and share this Cabbage Soup recipe from F...,"""https://img.sndimg.com/food/image/upload/w_55...",...,0.4,0.1,0.0,959.3,25.1,4.8,17.7,4.3,4.0,"c(""Mix everything together and bring to a boil..."


In [4]:
columns = [
    'RecipeId', 'Name', 'CookTime', 'PrepTime', 'TotalTime',
    'RecipeIngredientParts', 'Calories', 'FatContent', 'SaturatedFatContent',
    'CholesterolContent', 'SodiumContent', 'CarbohydrateContent',
    'FiberContent', 'SugarContent', 'ProteinContent', 'RecipeServings', 'Keywords', 
    'Images'
]
data = data[columns]

In [5]:
def extract_quoted_strings(s):
    return re.findall(r'"([^"]*)"', s)

def clean_image_url(url_string):
    if isinstance(url_string, str):
        urls = extract_quoted_strings(url_string)
        if urls:
            return urls[0] 
    return None

data['Images'] = data['Images'].apply(clean_image_url)

In [6]:
def preprocess_data(df):
    df = df.drop_duplicates(subset=['RecipeId'])
    
    # Improved time conversion function
    def convert_time(time_str):
        if pd.isna(time_str) or time_str == '': 
            return 0
            
        try:            
            time_str = str(time_str).replace('PT', '')
            
            hours = 0
            minutes = 0
            
            if 'H' in time_str:
                hours_part = time_str.split('H')[0]
                hours = int(hours_part) if hours_part else 0
                time_str = time_str.split('H')[1] if 'H' in time_str else time_str
                
            if 'M' in time_str:
                minutes_part = time_str.split('M')[0]
                minutes = int(minutes_part) if minutes_part else 0
                
            return hours * 60 + minutes
        except:
            return 0  
    
    for col in ['CookTime', 'PrepTime', 'TotalTime']:
        df[col] = df[col].apply(convert_time)
    
    # Normalize nutritional values per serving
    nut_cols = ['Calories', 'FatContent', 'SaturatedFatContent', 'CholesterolContent',
               'SodiumContent', 'CarbohydrateContent', 'FiberContent', 'SugarContent', 'ProteinContent']
    
    for col in nut_cols:
        df[col] = df[col] / df['RecipeServings'].replace(0, 1)  
    
    return df

processed_data = preprocess_data(data)

In [7]:
feature_cols = ['Calories', 'ProteinContent', 'CarbohydrateContent', 'CookTime']
features = processed_data[feature_cols].values
scaler = MinMaxScaler()
normalized_features = scaler.fit_transform(features)

In [8]:
def prepare_training_data(features, data):
    protein_high = data['ProteinContent'] > data['ProteinContent'].median()
    carb_low = data['CarbohydrateContent'] < data['CarbohydrateContent'].median()
    y = (protein_high & carb_low).astype(int)
    return train_test_split(features, y, test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = prepare_training_data(normalized_features, processed_data)

In [9]:
def build_model():
    model = keras.Sequential([
        keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [10]:
model = build_model()
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val))

/Users/adikazahran/anaconda3/envs/ganenv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
3545/3545 ━━━━━━━━━━━━━━━━━━━━ 4s 886us/step - accuracy: 0.8181 - loss: 0.4701 - val_accuracy: 0.8764 - val_loss: 0.2738
Epoch 2/20
3545/3545 ━━━━━━━━━━━━━━━━━━━━ 2s 662us/step - accuracy: 0.9078 - loss: 0.2292 - val_accuracy: 0.9581 - val_loss: 0.1161
Epoch 3/20
3545/3545 ━━━━━━━━━━━━━━━━━━━━ 3s 754us/step - accuracy: 0.9543 - loss: 0.1213 - val_accuracy: 0.9708 - val_loss: 0.0775
Epoch 4/20
3545/3545 ━━━━━━━━━━━━━━━━━━━━ 3s 711us/step - accuracy: 0.9679 - loss: 0.0839 - val_accuracy: 0.9837 - val_loss: 0.0555
Epoch 5/20
3545/3545 ━━━━━━━━━━━━━━━━━━━━ 3s 859us/step - accuracy: 0.9710 - loss: 0.0729 - val_accuracy: 0.9804 - val_loss: 0.0542
Epoch 6/20
3545/3545 ━━━━━━━━━━━━━━━━━━━━ 3s 733us/step - accuracy: 0.9768 - loss: 0.0597 - val_accuracy: 0.9834 - val_loss: 0.0522
Epoch 7/20
3545/3545 ━━━━━━━━━━━━━━━━━━━━ 3s 772us/step - accuracy: 0.9791 - loss: 0.0535 - val_accuracy: 0.9820 - val_loss: 0.0452
Epoch 8/20
3545/3545 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step - accuracy: 0.9788 - l

In [11]:
def recommend_recipes(model, features, data, calorie_target, n=3):
    scores = model.predict(features)
    data['Score'] = scores
    
    # Get top matches within 20% of calorie target
    mask = (data['Calories'] >= calorie_target*0.8) & (data['Calories'] <= calorie_target*1.2)
    return data[mask].nlargest(n, 'Score')

def get_user_input():
    user_data = {
        'gender': input("Jenis kelamin (Male/Female): "),
        'age': int(input("Umur (tahun): ")),
        'height': float(input("Tinggi badan (cm): ")),
        'weight': float(input("Berat badan (kg): ")),
        'activity': int(input("Tingkat aktivitas fisik (1-5): ")),
        'goal': int(input("Tujuan (1: Maintain, 2: Mild Loss, 3: Weight Loss, 4: Extreme Loss, 5: Weight Gain): "))
    }
    meals_per_day = int(input("Jumlah waktu makan per hari (3-5): "))
    return {**user_data, 'meals_per_day': meals_per_day}

In [17]:
if __name__ == "__main__":
    try:
        print("=== Food Recommendation ===")
        user = get_user_input()
        
        # Calculate metrics
        height_m = user['height']/100
        bmi = user['weight'] / (height_m ** 2)
        bmi_category = ("Underweight" if bmi < 18.5 else
                       "Normal" if bmi < 25 else
                       "Overweight" if bmi < 30 else
                       "Obese") 
        
        # Calculate calorie needs
        bmr = (10*user['weight'] + 6.25*user['height'] - 5*user['age'] + 
              (5 if user['gender'].lower()=='male' else -161))
        activity_multiplier = [1.2, 1.375, 1.55, 1.725, 1.9][user['activity']-1]
        maintenance = bmr * activity_multiplier
        
        goal_adjustment = {1:0, 2:-250, 3:-500, 4:-750, 5:300}
        target_calories = maintenance + goal_adjustment[user['goal']]
        
        # Generate recommendations
        def get_recommendations(calorie_target):
            scores = model.predict(normalized_features)
            processed_data['Score'] = scores
            mask = (processed_data['Calories'] >= calorie_target*0.8) & \
                   (processed_data['Calories'] <= calorie_target*1.2)
            return processed_data[mask].nlargest(3, 'Score')
        
        breakfast = get_recommendations(target_calories*0.25)
        lunch = get_recommendations(target_calories*0.35)
        dinner = get_recommendations(target_calories*0.3)
        snack = get_recommendations(target_calories*0.1) if user['meals_per_day'] > 3 else None
        
        # Display results
        print("\n" + "="*50)
        print(f"BMI:\nNilai BMI: {bmi:.2f}\nKategori BMI: {bmi_category}")
        print(f"\nKebutuhan Kalori Harian:\nTotal Kalori: {target_calories:.0f} kkal")
        print("Pembagian Kalori:")
        print(f"Breakfast: {target_calories*0.25:.0f} kkal")
        print(f"Lunch: {target_calories*0.35:.0f} kkal")
        print(f"Dinner: {target_calories*0.3:.0f} kkal")
        if snack is not None:
            print(f"Snack: {target_calories*0.1:.0f} kkal")
        
        def print_meal(meal_name, meal_data):
            print(f"\n{meal_name} Menu:")
            for _, row in meal_data.iterrows():
                cuisine = re.findall(r'"([^"]*)"', str(row['Keywords']))
                cuisine = ', '.join(cuisine) if cuisine else "Unknown"
                print(f"- {row['Name']} (Cuisine: {cuisine})")
                print(f"  Kalori: {row['Calories']:.1f} kkal")
                print(f"  Protein: {row['ProteinContent']:.1f} g")
                if row['Images']:
                    display(HTML(f'<img src="{row["Images"]}" alt="Recipe Image" style="max-width: 300px; height: auto;"/><br>'))
                else:
                    display(HTML('<p><b>Gambar tidak ditemukan</b></p>'))

        print_meal("## 1. Breakfast", breakfast)
        print_meal("## 2. Lunch", lunch)
        print_meal("## 3. Dinner", dinner)
        if snack is not None:
            print_meal("## 4. Snack", snack)
            
    except Exception as e:
        print(f"\nError: {str(e)}")
        print("Pastikan semua input sudah benar dan coba lagi.")

=== Food Recommendation ===
4431/4431 ━━━━━━━━━━━━━━━━━━━━ 2s 433us/step
4431/4431 ━━━━━━━━━━━━━━━━━━━━ 2s 395us/step
4431/4431 ━━━━━━━━━━━━━━━━━━━━ 1s 290us/step
4431/4431 ━━━━━━━━━━━━━━━━━━━━ 1s 257us/step

BMI:
Nilai BMI: 35.29
Kategori BMI: Obese

Kebutuhan Kalori Harian:
Total Kalori: 2756 kkal
Pembagian Kalori:
Breakfast: 689 kkal
Lunch: 965 kkal
Dinner: 827 kkal
Snack: 276 kkal

## 1. Breakfast Menu:
- Pork Sirloin Roast With Thai Peanut Sauce (Slow Cooker) (Cuisine: Meat, Peanut Butter, Thai, Asian, Lactose Free, Free Of..., Weeknight, Small Appliance, Easy, From Scratch)
  Kalori: 688.6 kkal
  Protein: 77.2 g


- A@@ Kicking Porterhouse Steak (Cuisine: Meat, Very Low Carbs, High Protein, High In..., < 15 Mins, Easy)
  Kalori: 809.6 kkal
  Protein: 65.3 g


- Roast Chicken Breast (Bone In, Skin On) (Cuisine: Chicken, Poultry, Meat, Very Low Carbs, High Protein, High In..., Weeknight, < 30 Mins, Inexpensive)
  Kalori: 600.3 kkal
  Protein: 60.9 g



## 2. Lunch Menu:
- Chicken Breast Filled With Bacon &amp; Cheese (Cuisine: Chicken, Poultry, Meat, Australian, Very Low Carbs, High Protein, High In..., Savory, < 15 Mins, Beginner Cook, Stove Top, Easy, Inexpensive)
  Kalori: 1005.1 kkal
  Protein: 80.0 g


- Hackney's Inside-Out Burger (Cuisine: Lunch/Snacks, Cheese, Meat, Very Low Carbs, High Protein, Kid Friendly, High In..., < 15 Mins, Beginner Cook, Easy)
  Kalori: 1003.0 kkal
  Protein: 59.7 g


...

- A@@ Kicking Porterhouse Steak (Cuisine: Meat, Very Low Carbs, High Protein, High In..., < 15 Mins, Easy)
  Kalori: 809.6 kkal
  Protein: 65.3 g


- Super-Easy No-Boil Oven Ribs (Cuisine: Meat, Very Low Carbs, High Protein, High In..., Roast, Broil/Grill, Oven, Beginner Cook, < 4 Hours, Easy)
  Kalori: 759.4 kkal
  Protein: 54.6 g



## 4. Snack Menu:
- Smelly Tuna (Cuisine: Tuna, Very Low Carbs, High Protein, High In..., Microwave, < 15 Mins, Easy, Inexpensive)
  Kalori: 316.8 kkal
  Protein: 46.5 g


- Grilled Rosemary Pork Chops (Cuisine: Meat, Very Low Carbs, Lactose Free, High Protein, Egg Free, Healthy, Free Of..., High In..., Savory, < 15 Mins, Easy)
  Kalori: 307.1 kkal
  Protein: 50.0 g


- Reheating Cooked Shellfish (Cuisine: High Protein, Healthy, High In..., < 30 Mins, Steam, Stove Top, Easy)
  Kalori: 330.3 kkal
  Protein: 65.9 g
